Import statements in Python

In [2]:
import cv2
import os
from __future__ import print_function # Python 2/3 compatibility
import cv2 # Import the OpenCV library
import numpy as np # Import Numpy library
from statistics import mean

### Quick validation of velocity of one pixel

In [125]:
# Load an color image in grayscale
img = cv2.imread('frame0.jpg',0)

In [126]:
cv2.imshow('image',img)

In [45]:
# 88 x 37 across 1 cm with tick marks for every 1mm
# so 1mm is 
pixPerMM = 8.8 # pixel distance per mm

# do this more systematically... take the average of the brightness
# of the ruler!

In [72]:
# iphone 6 fps with slo-mo
fps = 240 # frames per second

In [73]:
# hand calculation - brief
# looking at frames: 123 to 131
numFrames = 131-123
heightPix = 1080 #from iphone settings
pixPerMM = 8.8 # pixel distance per mm

In [74]:
# mm per second
seconds = numFrames/fps 
distance = heightPix/pixPerMM
velocity = distance/seconds

In [76]:
print(seconds)

0.03333333333333333


In [77]:
print(distance) # around 4.8 inches

122.72727272727272


In [78]:
print(velocity) #mm per second

3681.8181818181815


From a quick validation test, we received around 3.681 m/s for the particle falling. Now, let us write functions in order for automatic distance calibration and additionally object detection and finding the average velocity from the centroids detected.

## Distance Calibration

To do accurate distance calibration, first position a ruler to where you expect the particles to drop. You want the ruler to have definite lines at each mm mark and put the ruler was perpendicular as possible to the surface of the table.

We recommend to take a quick 5 second video with the video stationary. We will be using one of the frames in order to calculate the pixel/mm ratio.

Please save this calibration test as a mp4 file and save into the same directory as the script.

In [127]:
nameCalibrationFile = "calibration_test.mp4"
c=cv2.VideoCapture(nameCalibrationFile)
c.isOpened()

True

In [128]:
# Function to extract frames
def FrameCapture(path):
      
    # Path to video file
    vidObj = cv2.VideoCapture(path)
  
    # Used as counter variable
    count = 0
  
    # checks whether frames were extracted
    success = 1
  
    while success:
  
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()
  
        # Saves the frames with frame-count
        # TODO: need to save into directory
        cv2.imwrite("frame%d.jpg" % count, image)
  
        count += 1
    
    # TODO: have a break statement

  
# Driver Code
if __name__ == '__main__':
  
    # Calling the function
    FrameCapture("calibration_test.mp4")

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgcodecs/src/loadsave.cpp:753: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


Import slow-mo motion video and keep track of velocity

In [46]:
c=cv2.VideoCapture("pixel_test_velocity.mp4")
c.isOpened()

True

In [47]:
if __name__ == '__main__':
  
    # Calling the function
    FrameCapture("pixel_test_velocity.mp4")

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgcodecs/src/loadsave.cpp:753: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


## Object detection

In [ ]:
test_video_dir = '/Users/yoo-jin/git/arcticiceclinic/ImageProcessing/pixel_test_velocity.mp4'

In [51]:
def numFramesVideo(test_video_dir):
    vidcapture = cv2.VideoCapture(test_video_dir)
    fps = vidcapture.get(cv2.CAP_PROP_FPS)
    totalNoFrames = vidcapture.get(cv2.CAP_PROP_FRAME_COUNT);
    durationInSeconds = float(totalNoFrames) / float(fps)
    return totalNoFrames, durationInSeconds

In [56]:
frames, seconds = numFramesVideo(test_video_dir)
print(frames, seconds)

463.0 15.433333333333334


In [81]:
#!/usr/bin/env python

def object_detection(threshold, video_dir, noFrames):
    """
    Input 
    threshold: pixel distance to keep track of for height and width
    video_dir: directory path to video that is wanted to be analyzed
    
    Output 
    distances (list of tuples): the captured position of the centroid 
    of the bounding box
    """
    distances = []
    i = 0
    
    # cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(video_dir)
 
    # Create the background subtractor object
    fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
 
    while(True):
        i = i + 1
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        # Use every frame to calculate the foreground mask and update
        # the background
        fg_mask = fgbg.apply(frame)
 
        # Remove salt and pepper noise with a median filter
        fg_mask = cv2.medianBlur(fg_mask, 5) 
         
        # Threshold the image to make it either black or white
        thresh, fg_mask = cv2.threshold(fg_mask,127,255,cv2.THRESH_BINARY)
 
        # Find the index of the largest contour and draw bounding box
        contours, hierarchy = cv2.findContours(fg_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2:]
        areas = [cv2.contourArea(c) for c in contours]
 
        # If there are no countours
        if len(areas) < 1:
 
            # Display the resulting frame
            cv2.imshow('frame',frame)
 
            # If "q" is pressed on the keyboard, 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
 
            # Go to the top of the while loop
            continue
 
        else:
            # Find the largest moving object in the image
            max_index = np.argmax(areas)
            
        # Draw the bounding box
        count = contours[max_index]
        x,y,w,h = cv2.boundingRect(count)
        
        # setting a threshold
        if w>threshold and h>threshold: 
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
            # Draw circle in the center of the bounding box
            x2 = x + int(w/2)
            y2 = y + int(h/2)
            
            # Drawing a circle on any image
            cv2.circle(frame,(x2,y2),2,(0,0,255),-1)
            # Note that the -1 fills in the circle

            # Print the centroid coordinates
            text = "x: " + str(x2) + ", y: " + str(y2)
            cv2.putText(frame, text, (x2, y2),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 4)
            
            distances.append((x2, y2))
            #print(distances)
        # Display the resulting frame
        cv2.imshow('frame',frame)
        
        if i == noFrames: 
            return distances
 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # Close down the video stream
    cap.release()
    cv2.destroyAllWindows()

In [83]:
test_video_dir = '/Users/yoo-jin/git/arcticiceclinic/ImageProcessing/pixel_test_velocity.mp4'

if __name__ == '__main__':
    #print(__doc__)
    distances = object_detection(10, test_video_dir, frames)

In [84]:
print(distances)

[(115, 40), (114, 108), (114, 176), (113, 243), (112, 311), (458, 39), (457, 118), (457, 216), (458, 311), (459, 404), (459, 499), (460, 589), (459, 682), (459, 774), (460, 865), (461, 943), (1380, 68), (1383, 163), (1384, 272), (1388, 386), (1396, 524), (1400, 650), (1403, 778), (1405, 892), (1424, 576), (1428, 682), (1433, 793), (1440, 903), (1312, 398), (1313, 466), (1314, 532), (1315, 598), (1316, 661), (1318, 723), (1319, 784), (1321, 846), (1323, 906), (1325, 955), (1400, 321), (1402, 375), (1409, 436), (1411, 491), (1413, 545), (1414, 599), (1416, 653), (1419, 707), (1421, 761), (1422, 815), (1428, 900), (1422, 886), (1423, 935), (1180, 458), (1181, 498), (1183, 538), (1184, 579), (1185, 618), (1185, 659), (1186, 699), (1187, 739), (1611, 11), (1618, 60), (1624, 151), (1632, 248), (1638, 350), (1644, 461), (1652, 567), (1659, 677), (1666, 790), (1671, 899), (1677, 955), (1485, 202), (1488, 271), (1491, 341), (1495, 410), (1499, 480), (1502, 551), (1506, 624), (1509, 694), (1513,

## Calculating velocity

In [87]:
test = [(1181, 498), (1183, 538)]
for j in range(len(test)):
    print(test[j][1])

498
538


In [122]:
def calculateVel(distances, pixMM, fps):
    """
    Inputs:
    distances: list of tuples containing the x and y positions
    pixMM: the pixel/mm received from distance calibration
    fps: frames per second
    Basic calculation: 
        pixel * (mm/pixel) * one frame * frame/second = mm/second
        
    Outputs:
    
    """
    velocityList = []
    for i in range(len(distances)-1):
        # pixel difference in y direction
        yPixdiff = abs(distances[i][1] - distances[i+1][1])
        
        if yPixdiff < 500: 
            velocity = yPixdiff*(1/pixMM)*1*fps
            velocityList.append(velocity)
            
        print(len(velocityList))

    return mean(velocityList)

In [123]:
pixMM = 8.8
fps = 240
aveVel = calculateVel(distances,pixMM,fps)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
15
16
17
18
19
20
21
22
23
24
25
26
26
27
28
29
30
31
32
33
34
35
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
55
56
57
58
59
60
61
62
63
64
65
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
175
176
177
178
179
180
181
182
183
184
185
186
186
187
188
189
190
191
192
193
194
195
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [124]:
print(aveVel)

2269.845779220779
